In [11]:
import pandas as pd
from tbdynamics.plotting import plot_model_vs_actual
import nevergrad as ng

# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.calibration.utils import get_bcm, load_targets
from multiprocessing import cpu_count

In [12]:
pd.options.plotting.backend = "plotly"

In [13]:
params = {
    "start_population_size": 1500000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
covid_effects = {
    'detection_reduction':True,
    'contact_reduction':False
}
bcm = get_bcm(params, covid_effects)

In [14]:
# Set the number of workers for parallel optimization
orunner = optimize_model(bcm, opt_class=ng.optimizers.TwoPointsDE, num_workers=cpu_count())
for i in range(8):
    rec = orunner.minimize(1000)
mle_params = rec.value[1]
mle_params

{'contact_rate': 0.020881577303022834,
 'rr_infection_latent': 0.18776489979274727,
 'rr_infection_recovered': 0.19490592629660813,
 'progression_multiplier': 1.0153664093637496,
 'smear_positive_death_rate': 0.3858902413714194,
 'smear_negative_death_rate': 0.02358738911703359,
 'smear_positive_self_recovery': 0.23433193130517802,
 'smear_negative_self_recovery': 0.1329701693296454,
 'screening_scaleup_shape': 0.3258591369260496,
 'screening_inflection_time': 1999.098829229279,
 'time_to_screening_end_asymp': 1.5663898354713988,
 'contact_reduction': 0.03211500892904105,
 'detection_reduction': 0.19291283004397197,
 'notif_dispersion': 4566.48343675673,
 'prev_dispersion': 20.72513277554215}

In [15]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets()

In [16]:
plot_model_vs_actual(
    derived_df_0, targets['total_population'], "total_population", "Population", "Modelled vs Data"
)

In [17]:
plot_model_vs_actual(derived_df_0, targets['adults_prevalence_pulmonary_target'], 'adults_prevalence_pulmonary', 'Infectious prevalence', 'Modelled vs Estimation from 2017 prevalence survey')

In [18]:
plot_model_vs_actual(
    derived_df_0, targets['incidence_target'], "incidence", "Incidence", "Modelled vs Data"
)

In [19]:
plot_model_vs_actual(
    derived_df_0, targets['notification'], "notification", "Notifications", "Modelled vs Data"
)

In [20]:
derived_df_0['detection_rate'].plot()